In [1]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [2]:
import torch
import pandas as pd
from collections import Counter
import random

class Dataset():
    def __init__(
        self,sequence_length
        #args,
    ):
        #self.args = args
        self.sequence_length = sequence_length
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]
    
    def load_words(self):
        train_df = pd.read_csv('human_Shakespeare.csv')
        text = train_df['text'].str.cat(sep=' ')

        return text.split(' ')      
       
    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        #return len(self.words_indexes) - self.args.sequence_length
        return len(self.words_indexes) - self.sequence_length
    def __getitem__(self, index):
        return (
            #torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),
            #torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1])
        )

In [10]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader

def train(dataset, model, sequence_length,batch_size,max_epochs):
    model.train()

    dataloader = DataLoader(dataset, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(max_epochs):
        #state_h, state_c = model.init_state(args.sequence_length)
        state_h, state_c = model.init_state(sequence_length)
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [4]:
def predict(dataset, model, text, next_words=500):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        #x = torch.tensor([[dataset.word_to_index[w] for w in words]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p) #p is probability distribution of words
        words.append(dataset.index_to_word[word_index])
        
    return words

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--max-epochs', type=int, default=10)
# parser.add_argument('--batch-size', type=int, default=256)
# parser.add_argument('--sequence-length', type=int, default=4)
# args = parser.parse_args()

dataset = Dataset(sequence_length=4)
model = Model(dataset)

train(dataset, model,sequence_length=4,batch_size=256,max_epochs=10)
#print(predict(dataset, model, text='Knock knock. Whos there?'))

In [ ]:
torch.save(model,'lstm_shakespeare.pt')

# Prediction

In [5]:
import torch

model = torch.load('lstm_shakespeare.pt')

In [6]:
data = pd.read_csv('human_Shakespeare.csv')
dataset = Dataset(sequence_length=4)

In [ ]:
dataset = Dataset(sequence_length=4)
import random
  
from random import randint
predicted_words = []
for i in range(500):
  sample_text = data['text'][randint(0,200)] # select a random row for word generation
  words = predict(dataset, model, text=sample_text, next_words=500)
  prediction = ' '.join(words)  
  predicted_words.append(prediction)
  

In [ ]:
df = pd.DataFrame(predicted_words,columns=['text'])
df.to_csv('LSTM_Generation.csv',index=False)

# Sample prediction

In [13]:
generate = predict(dataset, model, text='Well, God give thee the spirit of persuasion and him the ears',next_words=15)
print(' '.join(generate))

Well, God give thee the spirit of persuasion and him the ears Saint We Half when you all slave, A host, of illusions: and dinner: A experience
